In [1]:
import requests
import folium
import pandas as pd
from folium import IFrame

# Fetch the data from the API for house prices
url_houseprice = 'http://127.0.0.1:9090/airquality/houseprice'
response_houseprice = requests.get(url_houseprice)
data_houseprice = response_houseprice.json()

# Convert JSON data to a DataFrame
df_houseprice = pd.json_normalize(data_houseprice)

# Remove rows with NaN values in 'average_price'
df_houseprice = df_houseprice.dropna(subset=['average_price'])

# Create a Folium map centered on a specific location
m = folium.Map(width=900, height=600, location=[-37.8014, 144.9958], zoom_start=10, tiles='OpenStreetMap')

# Define a function to create styled HTML for the popups
def create_popup_html(row):
    html = f"""
    <div style="font-family: Arial; font-size: 14px;">
        <h4>{row['site_name']}</h4>
        <p><strong>Health Advice:</strong> <span style="color: {row['health_advice_color']};">{row['health_advice'] if not row['health_advice'] is None else '---'}</span></p>
        <p><strong>Value:</strong> {row['value'] if not pd.isna(row['value']) else '---'} {row['unit'] if not row['unit'] is None else ''}</p>
        <p><strong>Average House Price:</strong> ${int(row['average_price']):,}</p>
    </div>
    """
    return html

# Iterate over the DataFrame and add markers to the map
for index, row in df_houseprice.iterrows():
    popup_html = create_popup_html(row)
    iframe = IFrame(popup_html, width=200, height=150)
    popup = folium.Popup(iframe, max_width=300)
    
    folium.Marker(
        location=[row['site_location.latitude'], row['site_location.longitude']],
        popup=popup,
        tooltip=f"<strong>{row['site_name']}</strong>",
        icon=folium.Icon(color="blue", icon="info-sign"),
    ).add_to(m)

# Display the map
m
